In [1]:
!pip install numpy==1.15.0
!pip install numba==0.47.0
!pip install opencv-python==3.3.0.10
!pip install scipy==1.1.0
!pip install scikit-learn==0.19.1
!pip install scikit-image==0.13.1
!pip install Pillow==5.0.0

In [0]:

import cv2
from scipy.misc import imresize
from scipy.signal import convolve2d
import numpy as np
from math import atan2, floor, pi, ceil, isnan
import numba as nb
import os

IMG_EXTENSIONS = ['.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',]

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def make_dataset(dir):
    images = []
    assert os.path.isdir(dir), '%s is not a valid directory' % dir
    for root, _, fnames in sorted(os.walk(dir)):
        for fname in fnames:
            if is_image_file(fname):
                path = os.path.join(root, fname)
                images.append(path)
    return images

# Python opencv library (cv2) cv2.COLOR_BGR2YCrCb has different parameters with MATLAB color convertion.
# In order to have a fair comparison with the benchmark, we wrote these functions by ourselves.
def BGR2YCbCr(im):
    mat = np.array([[24.966, 128.553, 65.481],[112, -74.203, -37.797], [-18.214, -93.786, 112]])
    mat = mat.T
    offset = np.array([[[16, 128, 128]]])
    if im.dtype == 'uint8':
        mat = mat/255
        out = np.dot(im,mat) + offset
        out = np.clip(out, 0, 255)
        out = np.rint(out).astype('uint8')
    elif im.dtype == 'float':
        mat = mat/255
        offset = offset/255
        out = np.dot(im, mat) + offset
        out = np.clip(out, 0, 1)
    else:
        assert False
    return out

def YCbCr2BGR(im):
    mat = np.array([[24.966, 128.553, 65.481],[112, -74.203, -37.797], [-18.214, -93.786, 112]])
    mat = mat.T
    mat = np.linalg.inv(mat)
    offset = np.array([[[16, 128, 128]]])
    if im.dtype == 'uint8':
        mat = mat * 255
        out = np.dot((im - offset),mat)
        out = np.clip(out, 0, 255)
        out = np.rint(out).astype('uint8')
    elif im.dtype == 'float':
        mat = mat * 255
        offset = offset/255
        out = np.dot((im - offset),mat)
        out = np.clip(out, 0, 1)
    else:
        assert False
    return out

def im2double(im):
    if im.dtype == 'uint8':
        out = im.astype('float') / 255
    elif im.dtype == 'uint16':
        out = im.astype('float') / 65535
    elif im.dtype == 'float':
        out = im
    else:
        assert False
    out = np.clip(out, 0, 1)
    return out

def Gaussian2d(shape=(3,3),sigma=0.5):
    """
    2D gaussian mask - should give the same result as MATLAB's
    fspecial('gaussian',[shape],[sigma])
    from https://stackoverflow.com/questions/17190649/how-to-obtain-a-gaussian-filter-in-python

    """
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

def modcrop(im,modulo):
    shape = im.shape
    size0 = shape[0] - shape[0] % modulo
    size1 = shape[1] - shape[1] % modulo
    if len(im.shape) == 2:
        out = im[0:size0, 0:size1]
    else:
        out = im[0:size0, 0:size1, :]
    return out

def Prepare(im, patchSize, R):
    patchMargin = floor(patchSize/2)
    H, W = im.shape
    imL = imresize(im, 1 / R, interp='bicubic')
    # cv2.imwrite('Compressed.jpg', imL, [int(cv2.IMWRITE_JPEG_QUALITY), 85])
    # imL = cv2.imread('Compressed.jpg')
    # imL = imL[:,:,0]   # Optional: Compress the image
    imL = imresize(imL, (H, W), interp='bicubic')
    imL = im2double(imL)
    im_LR = imL
    return im_LR

def is_greyimage(im):
    x = abs(im[:,:,0]-im[:,:,1])
    y = np.linalg.norm(x)
    if y==0:
        return True
    else:
        return False

@nb.jit(nopython=True, parallel=True)
def Grad(patchX,patchY,weight):
    gx = patchX.ravel()
    gy = patchY.ravel()
    G = np.vstack((gx,gy)).T
    x0 = np.dot(G.T,weight)
    x = np.dot(x0, G)
    w,v = np.linalg.eig(x)
    index= w.argsort()[::-1]
    w = w[index]
    v = v[:,index]
    lamda = w[0]
    u = (np.sqrt(w[0]) - np.sqrt(w[1]))/(np.sqrt(w[0]) + np.sqrt(w[1]) + 0.00000000000000001)
    return lamda,u

@nb.jit(nopython=True, parallel=True)
def HashTable(patchX,patchY,weight, Qangle,Qstrength,Qcoherence,stre,cohe):
    assert (len(stre)== Qstrength-1) and (len(cohe)==Qcoherence-1),"Quantization number should be equal"
    gx = patchX.ravel()
    gy = patchY.ravel()
    G = np.vstack((gx,gy)).T
    x0 = np.dot(G.T,weight)
    x = np.dot(x0, G)
    w,v = np.linalg.eig(x)
    index= w.argsort()[::-1]
    w = w[index]
    v = v[:,index]
    theta = atan2(v[1,0], v[0,0])
    if theta<0:
        theta = theta+pi
    theta = floor(theta/(pi/Qangle))
    lamda = w[0]
    u = (np.sqrt(w[0]) - np.sqrt(w[1]))/(np.sqrt(w[0]) + np.sqrt(w[1]) + 0.00000000000000001)
    if isnan(u):
        u=1
    if theta>Qangle-1:
        theta = Qangle-1
    if theta<0:
        theta = 0
    lamda = np.searchsorted(stre,lamda)
    u = np.searchsorted(cohe,u)
    return theta,lamda,u

@nb.jit(nopython=True, parallel=True)
def Gaussian_Mul(x,y,wGaussian):
    result = np.zeros((x.shape[0], x.shape[1], y.shape[2]))
    for i in range(x.shape[0]):
        # inter = np.matmul(x[i], wGaussian)
        # result[i] = np.matmul(inter,y[i])
        inter = np.dot(x[i], wGaussian)
        result[i] = np.dot(inter, y[i])
    return result

def CT_descriptor(im):
    H, W = im.shape
    windowSize = 3
    Census = np.zeros((H, W))
    CT = np.zeros((H, W, windowSize, windowSize))
    C = np.int((windowSize-1)/2)
    for i in range(C,H-C):
        for j in range(C, W-C):
            cen = 0
            for a in range(-C, C+1):
                for b in range(-C, C+1):
                    if not (a==0 and b==0):
                        if im[i+a, j+b] < im[i, j]:
                            cen += 1
                            CT[i, j, a+C,b+C] = 1
            Census[i, j] = cen
    Census = Census/8
    return Census, CT

def Blending1(LR, HR):
    H,W = LR.shape
    H1,W1 = HR.shape
    assert H1==H and W1==W
    Census,CT = CT_descriptor(LR)
    blending1 = Census*HR + (1 - Census)*LR
    return blending1

def Blending2(LR, HR):
    H,W = LR.shape
    H1,W1 = HR.shape
    assert H1==H and W1==W
    Census1, CT1 = CT_descriptor(LR)
    Census2, CT2 = CT_descriptor(HR)
    weight = np.zeros((H, W))
    x = np.zeros(( 3, 3))
    for i in range(H):
        for j in range(W):
            x  = np.absolute(CT1[i,j]-CT2[i,j])
            weight[i, j] = x.sum()
    weight = weight/weight.max()
    blending2 = weight * LR + (1 - weight) * HR
    return blending2

def Backprojection(LR, HR, maxIter):
    H, W = LR.shape
    H1, W1 = HR.shape
    w = Gaussian2d((5,5), 10)
    w = w**2
    w = w/sum(np.ravel(w))
    for i in range(maxIter):
        im_L = imresize(HR, (H, W), interp='bicubic', mode='F')
        imd = LR - im_L
        im_d = imresize(imd, (H1, W1), interp='bicubic', mode='F')
        HR = HR + convolve2d(im_d, w, 'same')
    return HR

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

def Dog1(im):
    sigma = 0.85
    alpha = 1.414
    r = 15
    ksize = (3, 3)
    G1 = cv2.GaussianBlur(im, ksize, sigma)
    Ga1 = cv2.GaussianBlur(im, ksize, sigma*alpha)
    D1 = cv2.addWeighted(G1, 1+r, Ga1, -r, 0)

    G2 = cv2.GaussianBlur(Ga1, ksize, sigma)
    Ga2 = cv2.GaussianBlur(Ga1, ksize, sigma*alpha)
    D2 = cv2.addWeighted(G2, 1+r, Ga2, -r, 0)

    G3 = cv2.GaussianBlur(Ga2, ksize, sigma)
    Ga3 = cv2.GaussianBlur(Ga2, ksize, sigma * alpha)
    D3 = cv2.addWeighted(G3, 1+r, Ga3, -r, 0)

    B1 = Blending1(im, D3)
    B1 = Blending1(im, B1)
    B2 = Blending1(B1, D2)
    B2 = Blending1(im, B2)
    B3 = Blending1(B2, D1)
    B3 = Blending1(im, B3)

    output = B3

    return output

def Getfromsymmetry1(V, patchSize, t1, t2):
    V_sym = np.zeros((patchSize*patchSize,patchSize*patchSize))
    for i in range(1, patchSize*patchSize+1):
        for j in range(1, patchSize*patchSize+1):
            y1 = ceil(i/patchSize)
            x1 = i-(y1-1)*patchSize
            y2 = ceil(j/patchSize)
            x2 = j-(y2-1)*patchSize
            if (t1 == 1) and (t2 == 0):
                ig = patchSize * x1 + 1 - y1
                jg = patchSize * x2 + 1 - y2
                V_sym[ig - 1, jg - 1] = V[i - 1, j - 1]
            elif (t1 == 2) and (t2 == 0):
                x = patchSize + 1 - x1
                y = patchSize + 1 - y1
                ig = (y - 1) * patchSize + x
                x = patchSize + 1 - x2
                y = patchSize + 1 - y2
                jg = (y - 1) * patchSize + x
                V_sym[ig - 1, jg - 1] = V[i - 1, j - 1]
            elif (t1 == 3) and (t2 == 0):
                x = y1
                y = patchSize + 1 - x1
                ig =(y - 1) * patchSize + x
                x = y2
                y = patchSize + 1 - x2
                jg = (y - 1) * patchSize + x
                V_sym[ig - 1, jg - 1] = V[i - 1, j - 1]
            elif (t1 == 0) and (t2 == 1):
                x = patchSize + 1 - x1
                y = y1
                ig =(y - 1) * patchSize + x
                x = patchSize + 1 - x2
                y = y2
                jg = (y - 1) * patchSize + x
                V_sym[ig - 1, jg - 1] = V[i - 1, j - 1]
            elif (t1 == 1) and (t2 == 1):
                x0 = patchSize + 1 - x1
                y0 = y1
                x = patchSize + 1 - y0
                y = x0
                ig =(y - 1) * patchSize + x
                x0 = patchSize + 1 - x2
                y0 = y2
                x = patchSize + 1 - y0
                y = x0
                jg = (y - 1) * patchSize + x
                V_sym[ig - 1, jg - 1] = V[i - 1, j - 1]
            elif (t1 == 2) and (t2 == 1):
                x0 = patchSize + 1 - x1
                y0 = y1
                x = patchSize + 1 - x0
                y = patchSize + 1 - y0
                ig =(y - 1) * patchSize + x
                x0 = patchSize + 1 - x2
                y0 = y2
                x = patchSize + 1 - x0
                y = patchSize + 1 - y0
                jg = (y - 1) * patchSize + x
                V_sym[ig - 1, jg - 1] = V[i - 1, j - 1]
            elif (t1 == 3) and (t2 == 1):
                x0 = patchSize + 1 - x1
                y0 = y1
                x = y0
                y = patchSize + 1 - x0
                ig =(y - 1) * patchSize + x
                x0 = patchSize + 1 - x2
                y0 = y2
                x = y0
                y = patchSize + 1 - x0
                jg = (y - 1) * patchSize + x
                V_sym[ig - 1, jg - 1] = V[i - 1, j - 1]
            else:
                assert False
    return V_sym

def Getfromsymmetry2(V, patchSize, t1, t2):
    Vp = np.reshape(V, (patchSize, patchSize))
    V1 = np.rot90(Vp, t1)
    if t2 == 1:
        V1 = np.flip(V1, 1)
    V_sym = np.ravel(V1)
    return V_sym

# Quantization procedure to get the optimized strength and coherence boundaries
@nb.jit(nopython=True, parallel=True)
def QuantizationProcess (im_GX, im_GY,patchSize, patchNumber,w , quantization):
    H, W = im_GX.shape
    for i1 in range(H-2*floor(patchSize/2)):
        for j1 in range(W-2*floor(patchSize/2)):
            idx = (slice(i1,(i1+2*floor(patchSize/2)+1)),slice(j1,(j1+2*floor(patchSize/2)+1)))
            patchX = im_GX[idx]
            patchY = im_GY[idx]
            strength, coherence = Grad(patchX, patchY, w)
            quantization[patchNumber, 0] = strength
            quantization[patchNumber, 1] = coherence
            patchNumber += 1
    return quantization, patchNumber

# Training procedure for each image (use numba.jit to speed up)
@nb.jit(nopython=True, parallel=True)
def TrainProcess (im_LR, im_HR, im_GX, im_GY,patchSize, w, Qangle, Qstrength,Qcoherence, stre, cohe, R, Q, V, mark):
    H, W = im_HR.shape
    for i1 in range(H-2*floor(patchSize/2)):
        for j1 in range(W-2*floor(patchSize/2)):
            idx1 = (slice(i1,(i1+2*floor(patchSize/2)+1)),slice(j1,(j1+2*floor(patchSize/2)+1)))
            patch = im_LR[idx1]
            patchX = im_GX[idx1]
            patchY = im_GY[idx1]
            theta,lamda,u=HashTable(patchX, patchY, w, Qangle, Qstrength,Qcoherence, stre, cohe)
            patch1 = patch.ravel()
            patchL = patch1.reshape((1,patch1.size))
            t = (i1 % R) * R +(j1 % R)
            j = theta * Qstrength * Qcoherence + lamda * Qcoherence + u
            tx = np.int(t)
            jx = np.int(j)
            A = np.dot(patchL.T, patchL)
            Q[tx,jx] += A
            b1=patchL.T * im_HR[i1+floor(patchSize/2),j1+floor(patchSize/2)]
            b = b1.reshape((b1.size))
            V[tx,jx] += b
            mark[tx,jx] = mark[tx,jx]+1
    return Q,V,mark



In [0]:
#!/usr/bin/env python3


import numpy as np
import numba as nb
import os
import cv2
import pickle
import time
from math import floor
# from Jalali-Lab-Implementation-of-RAISR-master.Functions import *

trainPath = '/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/testfile'

R = 4                           # Upscaling factor=2
patchSize = 11                  # Pacth Size=11
gradientSize = 9                # Gradient Size = 9
Qangle = 24                     # Quantization factor of angle =24
Qstrength = 3                   # Quantization factor of strength =3
Qcoherence = 7                  # Quantization factor of coherence =3
stre = np.zeros((Qstrength-1))  # Strength boundary
cohe = np.zeros((Qcoherence-1)) # Coherence boundary

Q = np.zeros((R*R, Qangle*Qstrength*Qcoherence, patchSize*patchSize, patchSize*patchSize))  # Eq.4
V = np.zeros((R*R, Qangle*Qstrength*Qcoherence, patchSize*patchSize))                       # Eq.5
h = np.zeros((R*R, Qangle*Qstrength*Qcoherence, patchSize*patchSize))
mark = np.zeros((R*R, Qangle*Qstrength*Qcoherence))                  # statical distribution of patch numbers in each bucket
w = Gaussian2d([patchSize, patchSize], 2)
w = w/w.max()
w = np.diag(w.ravel())                                               # Diagnal weighting matrix Wk in Algorithm 1

filelist = make_dataset(trainPath)

instance = 20000000                          # use 20000000 patches to get the Strength and coherence boundary
patchNumber = 0                              # patch number has been used
quantization = np.zeros((instance,2))        # quantization boundary
for image in filelist:
    print('\r', end='')
    print('' * 60, end='')
    print('\r Quantization: Processing '+ str(instance/2) + ' patches (' + str(200*patchNumber/instance) + '%)')
    im_uint8 = cv2.imread(image)
    if is_greyimage(im_uint8):
        im_uint8 = im_uint8[:,:,0]
    if len(im_uint8.shape)>2:
        im_ycbcr = BGR2YCbCr(im_uint8)
        im = im_ycbcr[:,:,0]
    else:
        im = im_uint8
    im = modcrop(im,R)
    im_LR = Prepare(im,patchSize,R)         # Prepare the cheap-upscaling images (optional: JPEG compression)
    im_GX,im_GY = np.gradient(im_LR)        # Calculate the gradient images
    quantization, patchNumber = QuantizationProcess (im_GX, im_GY,patchSize, patchNumber, w, quantization)  # get the strength and coherence of each patch
    if (patchNumber > instance/2):
        break

# uniform quantization of patches, get the optimized strength and coherence boundaries
quantization = quantization [0:patchNumber,:]
quantization = np.sort(quantization,axis=0)
for i in range(Qstrength-1):
    stre[i] = quantization[floor((i+1)*patchNumber/Qstrength),0]
for i in range(Qcoherence-1):
    cohe[i] = quantization[floor((i+1)*patchNumber/Qcoherence),1]

print('Begin to process images:')
imagecount = 1
for image in filelist:
    print('\r', end='')
    print('' * 60, end='')
    print('\r Processing ' + str(imagecount) +'/' + str(len(filelist))+ ' image ('   + image + ')')
    im_uint8 = cv2.imread(image)
    if is_greyimage(im_uint8):
        im_uint8 = im_uint8[:,:,0]
    if len(im_uint8.shape)>2:
        im_ycbcr = BGR2YCbCr(im_uint8)
        im = im_ycbcr[:,:,0]
    else:
        im = im_uint8
    im = modcrop(im,R)
    im_LR = Prepare(im,patchSize,R)
    im_HR = im2double(im)
    #im_HR = Dog1(im_HR)                # optional: sharpen the image
    im_GX,im_GY = np.gradient(im_LR)
    Q, V, mark = TrainProcess(im_LR, im_HR, im_GX, im_GY,patchSize, w, Qangle, Qstrength,Qcoherence, stre, cohe, R, Q, V, mark)  # get Q, V of each patch
    imagecount += 1

# optional: Using patch symmetry for nearly 8* more learning examples
# print('\r', end='')
# print(' ' * 60, end='')
# print('\r Using patch symmetry for nearly 8* more learning examples:')
# for i in range(Qangle):
#     for j in range(Qstrength*Qcoherence):
#         for r in range(R*R):
#             for t in range(1,8):
#                 t1 = t % 4
#                 t2 = floor(t / 4)
#                 Q1 = Getfromsymmetry1(Q[r, i * Qstrength * Qcoherence + j], patchSize, t1, t2)  # Rotate 90*t1 degree or flip t2
#                 V1 = Getfromsymmetry2(V[r, i * Qstrength * Qcoherence + j], patchSize, t1, t2)
#                 i1 = Qangle*t1/2 + i
#                 i1 = np.int(i1)
#                 if t2 == 1:
#                     i1 = Qangle -1 - i1
#                 while i1 >= Qangle:
#                     i1 = i1 - Qangle
#                 while i1 < 0:
#                     i1 = i1 + Qangle
#                 Q[r, i1 * Qstrength * Qcoherence + j] += Q1
#                 V[r, i1 * Qstrength * Qcoherence + j] += V1


print('Get the filter of RAISR:')
for t in range(R*R):
    print(t)
    for j in range(Qangle*Qstrength*Qcoherence):
        while(True):
            if(Q[t,j].sum()<100):
                break
            if(np.linalg.det(Q[t,j])<1):
                Q[t,j] = Q[t,j] + np.eye(patchSize*patchSize)* Q[t,j].sum()*0.000000005
            else:
                h[t,j] = np.linalg.inv(Q[t,j]).dot(V[t,j])         # Eq.2
                break

with open("filter"+str(R), "wb") as fp:
    pickle.dump(h, fp)

with open("Qfactor_str"+str(R), "wb") as sp:
    pickle.dump(stre, sp)

with open("Qfactor_coh"+str(R), "wb") as cp:
    pickle.dump(cohe, cp)


print('\r', end='')
print(' ' * 60, end='')
print('\rFinished.')



 Quantization: Processing 10000000.0 patches (0.0%)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:106: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
<ipython-input-2-5f5521ffa9f8>:360: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "<ipython-input-2-5f5521ffa9f8>", line 124:
@nb.jit(nopython=True, parallel=True)
def Grad(patchX,patchY,weight):
^

  strength, coherence = Grad(patchX, patchY, w)
/usr/local/lib/python3.6/dist-packages/numba/typed_passes.py:293: 

 Quantization: Processing 10000000.0 patches (0.60516%)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:106: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 Quantization: Processing 10000000.0 patches (1.21032%)
 Quantization: Processing 10000000.0 patches (1.81548%)
 Quantization: Processing 10000000.0 patches (2.42064%)
 Quantization: Processing 10000000.0 patches (3.0258%)
 Quantization: Processing 10000000.0 patches (3.63096%)
 Quantization: Processing 10000000.0 patches (4.23612%)
 Quantization: Processing 10000000.0 patches (4.84128%)
 Quantization: Processing 10000000.0 patches (5.44644%)
 Quantization: Processing 10000000.0 patches (6.0516%)
 Quantization: Processing 10000000.0 patches (6.65676%)
 Quantization: Processing 10000000.0 patches (7.26192%)
 Quantization: Processing 10000000.0 patches (7.86708%)
 Quantization: Processing 10000000.0 patches (8.47224%)
 Quantization: Processing 10000000.0 patches (9.0774%)
 Quantization: Processing 10000000.0 patches (9.68256%)
 Quantization: Processing 10000000.0 patches (10.28772%)
Begin to process images:
 Processing 1/18 image (/content/drive/My Drive/Jalali-Lab-Implementation-of-RAIS

<ipython-input-2-5f5521ffa9f8>:376: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "<ipython-input-2-5f5521ffa9f8>", line 139:
@nb.jit(nopython=True, parallel=True)
def HashTable(patchX,patchY,weight, Qangle,Qstrength,Qcoherence,stre,cohe):
^

  theta,lamda,u=HashTable(patchX, patchY, w, Qangle, Qstrength,Qcoherence, stre, cohe)
/usr/local/lib/python3.6/dist-packages/numba/typed_passes.py:293: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "<ipython-input-2-5f5521ffa9f8>", line 368:
@nb.jit(nopython=True, parallel=True)
de

 Processing 2/18 image (/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/testfile/0040.png)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:106: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 Processing 3/18 image (/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/testfile/0017.png)
 Processing 4/18 image (/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/testfile/0032.png)
 Processing 5/18 image (/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/testfile/0020.png)
 Processing 6/18 image (/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/testfile/0022.png)
 Processing 7/18 image (/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/testfile/0027.png)
 Processing 8/18 image (/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/testfile/0026.png)
 Processing 9/18 image (/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/testfile/0023.png)
 Processing 10/18 image (/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/testfile/0041.png)
 Processing 11/18 image (/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/testfile/0031.png)
 Processing 12/18

In [0]:
#!/usr/bin/env python3

"""
Implementation of RAISR in Python by Jalali-Lab  

[RAISR](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7744595) (Rapid and Accurate Image Super 
Resolution) is an image processing algorithm published by Google Research in 2016. With sufficient 
trainingData, consisting of low and high resolution image pairs, RAISR algorithm tries to learn
a set of filters which can be applied to an input image that is not in the training set, 
to produce a higher resolution version of it. The source code released here is the Jalali-Lab 
implementation of the RAISR algorithm in Python 3.x. The implementation presented here achieved 
performance results that are comparable to that presented in Google's research paper 
(with ± 0.1 dB in PSNR). 

Just-in-time (JIT) compilation employing JIT numba is used to speed up the Python code. A very 
parallelized Python code employing multi-processing capabilities is used to speed up the testing 
process. The code has been tested on GNU/Linux and Mac OS X 10.13.2 platforms. 

Author: Sifeng He, Jalali-Lab, Department of Electrical and Computer Engineering, UCLA

Copyright
---------
RAISR is developed in Jalali Lab at University of California, Los Angeles (UCLA).  
More information about the technique can be found in our group website: http://www.photonics.ucla.edu


"""

import numpy as np
import numba as nb
import os
import cv2
import warnings
import pickle
import time
from math import floor
from skimage.measure import compare_psnr
from scipy.misc import imresize
# from Functions import *
import matplotlib.image as mpimg
from multiprocessing import Pool

warnings.filterwarnings('ignore')

testPath = '/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/testing'

R = 4  # Upscaling factor=2 R = [ 2 3 4 ] originally 2
patchSize = 11  # Pacth Size=11
gradientSize = 9
Qangle = 24  # Quantization factor of angle =24
Qstrength = 3  # Quantization factor of strength =3
Qcoherence = 7  # Quantization factor of coherence =3

with open("filter"+str(R), "rb") as fp:
    h = pickle.load(fp)

with open("Qfactor_str"+str(R), "rb") as sp:
    stre = pickle.load(sp)

with open("Qfactor_coh"+str(R), "rb") as cp:
    cohe = pickle.load(cp)

filelist = make_dataset(testPath)

wGaussian = Gaussian2d([patchSize, patchSize], 2)
wGaussian = wGaussian/wGaussian.max()
wGaussian = np.diag(wGaussian.ravel())
imagecount = 1
patchMargin = floor(patchSize/2)
psnrRAISR = []
psnrBicubic = []
psnrBlending = []

def TestProcess(i):
    if (i < iteration - 1):
        offset_i = offset[i * batch:i * batch + batch]
        offset2_i = offset2[i * batch:i * batch + batch]
        grid = np.tile(gridon[..., None], [1, 1, batch]) + np.tile(offset_i, [patchSize, patchSize, 1])
    else:
        offset_i = offset[i * batch:im.size]
        offset2_i = offset2[i * batch:im.size]
        grid = np.tile(gridon[..., None], [1, 1, im.size - (iteration - 1) * batch]) + np.tile(offset_i,[patchSize, patchSize,1])
    f = im_LR.ravel()[grid]
    gx = im_GX.ravel()[grid]
    gy = im_GY.ravel()[grid]
    gx = gx.reshape((1, patchSize * patchSize, gx.shape[2]))
    gy = gy.reshape((1, patchSize * patchSize, gy.shape[2]))
    G = np.vstack((gx, gy))
    g1 = np.transpose(G, (2, 0, 1))
    g2 = np.transpose(G, (2, 1, 0))
    x = Gaussian_Mul(g1, g2, wGaussian)
    w, v = np.linalg.eig(x)
    idx = (-w).argsort()
    w = w[np.arange(np.shape(w)[0])[:, np.newaxis], idx]
    v = v[np.arange(np.shape(v)[0])[:, np.newaxis, np.newaxis], np.arange(np.shape(v)[1])[np.newaxis, :, np.newaxis], idx[:,np.newaxis,:]]
    thelta = np.arctan(v[:, 1, 0] / v[:, 0, 0])
    thelta[thelta < 0] = thelta[thelta < 0] + pi
    thelta = np.floor(thelta / (pi / Qangle))
    thelta[thelta > Qangle - 1] = Qangle - 1
    thelta[thelta < 0] = 0
    lamda = w[:, 0]
    u = (np.sqrt(w[:, 0]) - np.sqrt(w[:, 1])) / (np.sqrt(w[:, 0]) + np.sqrt(w[:, 1]) + 0.00000000000000001)
    lamda = np.searchsorted(stre, lamda)
    u = np.searchsorted(cohe, u)
    j = thelta * Qstrength * Qcoherence + lamda * Qcoherence + u
    j = j.astype('int')
    offset2_i = np.unravel_index(offset2_i, (H, W))
    t = ((offset2_i[0] - patchMargin) % R) * R + ((offset2_i[1] - patchMargin) % R)
    filtertj = h[t, j]
    filtertj = filtertj[:, :, np.newaxis]
    patch = f.reshape((1, patchSize * patchSize, gx.shape[2]))
    patch = np.transpose(patch, (2, 0, 1))
    result = np.matmul(patch, filtertj)
    return result


print('Begin to process images:')
for image in filelist:
    print('\r', end='')
    print('' * 60, end='')
    print('\r Processing ' + str(imagecount) + '/' + str(len(filelist)) + ' image (' + image + ')')
    im_uint8 = cv2.imread(image)
    im_mp = mpimg.imread(image)
    if len(im_mp.shape) == 2:
        im_uint8 = im_uint8[:,:,0]
    im_uint8 = modcrop(im_uint8, R)
    if len(im_uint8.shape) > 2:
        im_ycbcr = BGR2YCbCr(im_uint8)
        im = im_ycbcr[:, :, 0]
    else:
        im = im_uint8
    im_double = im2double(im)
    H, W = im.shape
    region = (slice(patchMargin, H - patchMargin), slice(patchMargin, W - patchMargin))
    start = time.time()
    imL = imresize(im_double, 1 / R, interp='bicubic', mode='F')
    im_bicubic = imresize(imL, (H, W), interp='bicubic', mode='F')
    im_bicubic = im_bicubic.astype('float64')
    im_bicubic = np.clip(im_bicubic, 0, 1)
    im_LR = np.zeros((H+patchSize-1,W+patchSize-1))
    im_LR[(patchMargin):(H+patchMargin),(patchMargin):(W+patchMargin)] = im_bicubic
    im_result = np.zeros((H, W))
    im_GX, im_GY = np.gradient(im_LR)
    index = np.array(range(im_LR.size)).reshape(im_LR.shape)
    offset = np.array(index[0:H, 0:W].ravel())
    offset2 = np.array(range(im.size))
    gridon = index[0:patchSize, 0:patchSize]
    batch = 2000
    iteration = ceil(im.size / batch + 0.000000000000001)
    im_result = np.array([])

    i = range(iteration)
    p = Pool()
    im_in = p.map(TestProcess, i)

    for i in range(iteration):
        im_result = np.append(im_result, im_in[i])

    im_result = im_result.reshape(H, W)
    im_result = np.clip(im_result, 0, 1)

    end = time.time()
    print(end - start)

    im_blending = Blending2(im_bicubic, im_result)
    # im_blending = Backprojection(imL, im_blending, 50) #Optional: Backprojection, which can slightly improve PSNR, especilly for large upscaling factor.
    im_blending = np.clip(im_blending, 0, 1)

    if len(im_uint8.shape) > 2:
        result_ycbcr = np.zeros((H, W, 3))
        result_ycbcr[:, :, 0] = im_blending * 255
        result_ycbcr[:, :, 1] = Prepare(im_ycbcr[:, :, 1], patchSize, R) * 255
        result_ycbcr[:, :, 2] = Prepare(im_ycbcr[:, :, 2], patchSize, R) * 255
        result_ycbcr = result_ycbcr[region].astype('uint8')
        result_RAISR = YCbCr2BGR(result_ycbcr)
    else:
        result_RAISR = im_result[region] * 255
        result_RAISR = result_RAISR.astype('uint8')

    im_result = im_result*255
    im_result = np.rint(im_result).astype('uint8')
    im_bicubic = im_bicubic*255
    im_bicubic = np.rint(im_bicubic).astype('uint8')
    im_blending = im_blending * 255
    im_blending = np.rint(im_blending).astype('uint8')

    PSNR_bicubic = compare_psnr(im[region], im_bicubic[region])
    PSNR_result = compare_psnr(im[region], im_result[region])
    PSNR_blending = compare_psnr(im[region], im_blending[region])
    PSNR_blending = max(PSNR_result, PSNR_blending)

    createFolder('/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/results/')
    cv2.imwrite('/content/drive/My Drive/Jalali-Lab-Implementation-of-RAISR-master/results/' + os.path.splitext(os.path.basename(image))[0] + '_result.bmp', result_RAISR)
    psnrRAISR.append(PSNR_result)
    psnrBicubic.append(PSNR_bicubic)
    psnrBlending.append(PSNR_blending)

    imagecount += 1


RAISR_psnrmean = np.array(psnrBlending).mean()
Bicubic_psnrmean = np.array(psnrBicubic).mean()


print('\r', end='')
print('' * 60, end='')
print('\r RAISR PSNR of '+ testPath +' is ' + str(RAISR_psnrmean))
print('\r Bicubic PSNR of '+ testPath +' is ' + str(Bicubic_psnrmean))



In [0]:
# from model import resolve_single
# import utils
# from utils import load_image, plot_sample

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image


def load_image(path):
    return np.array(Image.open(path))


def plot_sample(hr,res):
    plt.figure(figsize=(20, 10))

    images = [hr,res]
    # images=[r]
    titles = ['HR', f'SR (x{res.shape[0] // hr.shape[0]})']

    for i, (img,title) in enumerate(zip(images,titles)):
        plt.subplot(1, 2, i+1)
        plt.imshow(img)
        plt.title(title)
        plt.xticks([])
        plt.yticks([])

def resolve_and_plot(example,sr_image_path):
    hr = load_image(example)
    res = load_image(sr_image_path)
    # sr = resolve_single(model, lr)
    plot_sample(hr,res)
resolve_and_plot('/content/0009.png','/content/results/0009_result.bmp')
# resolve_and_plot('/content/0009.png')